In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
# Reading the csv file
raw_df = pd.read_csv("Iris.csv")
new_df = raw_df.drop('Id', axis=1)

In [3]:
print(new_df.Species.unique())
# Assigning numerical values to the "Species" coloumn
h = new_df['Species'].str.get_dummies("EOL")
new_df = new_df.merge(h, left_index=True, right_index=True)
#new_df['Species'] = new_df['Species'].map({'Iris-setosa':1, 'Iris-versicolor':0, 'Iris-virginica':-1})

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [4]:
# Splitting the dataset in train and test sets

def setosa():# Classifier: Iris-setosa
    train_x, test_x, train_y, test_y = train_test_split(new_df.drop(['Species','Iris-setosa', 'Iris-versicolor', 'Iris-virginica'],axis=1), new_df['Iris-setosa'], test_size=0.2, random_state=5)
    train_x = train_x.T
    test_x = test_x.T
    # To prevent dimensions of the form (m,)
    # instead we want of the form (m,n)
    train_y = pd.DataFrame(train_y).T
    test_y = pd.DataFrame(test_y).T
    # To obtain array
    train_y = train_y.to_numpy()
    test_y = test_y.to_numpy()
    return train_x, test_x, train_y, test_y

def versicolor():# Classifier: Iris-versicolor
    train_x, test_x, train_y, test_y = train_test_split(new_df.drop(['Species','Iris-setosa', 'Iris-versicolor', 'Iris-virginica'],axis=1), new_df['Iris-versicolor'], test_size=0.2, random_state=8)
    train_x = train_x.T
    test_x = test_x.T
    # To prevent dimensions of the form (m,)
    # instead we want of the form (m,n)
    train_y = pd.DataFrame(train_y).T
    test_y = pd.DataFrame(test_y).T
    # To obtain array
    train_y = train_y.to_numpy()
    test_y = test_y.to_numpy()
    return train_x, test_x, train_y, test_y

def virginica():# Classifier: Iris-virginica
    train_x, test_x, train_y, test_y = train_test_split(new_df.drop(['Species','Iris-setosa', 'Iris-versicolor', 'Iris-virginica'],axis=1), new_df['Iris-virginica'], test_size=0.2, random_state=15)
    train_x = train_x.T
    test_x = test_x.T
    # To prevent dimensions of the form (m,)
    # instead we want of the form (m,n)
    train_y = pd.DataFrame(train_y).T
    test_y = pd.DataFrame(test_y).T
    # To obtain array
    train_y = train_y.to_numpy()
    test_y = test_y.to_numpy()
    return train_x, test_x, train_y, test_y


In [5]:
def sigmoid(z):
    a = 1/(1+np.exp(-z))
    return a

def layer_size(X, Y): 
    n_x = X.shape[0]
    n_h = 4 # hard-coded single layer size
    n_y = Y.shape[0]
    
    return n_x, n_h, n_y

def initialize_parameters(X, Y):
    n_x, n_h, n_y = layer_size(X, Y)
    W1 = np.random.randn(n_h, n_x)
    b1 = np.zeros(shape=(n_h, 1))
    W2 = np.random.randn(n_y, n_h)
    b2 = np.zeros(shape=(n_y, 1))
    
    parameters = {'W1': W1,
                  'b1': b1,
                  'W2': W2,
                  'b2': b2}
    return parameters

def forward_prop(X, parameters):
    W2 = parameters['W2']
    b2 = parameters['b2']
    W1 = parameters['W1']
    b1 = parameters['b1']
        
    Z1 = np.dot(W1,X)+b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2,A1)+b2
    A2 = sigmoid(Z2)
    
    cache = {'Z1':Z1,
             'A1':A1,
             'Z2':Z2,
             'A2':A2}
    return A2, cache

def cost_function(A2, Y):
    #W1 = parameters['W1']
    #W2 = parameters['W2']
    
    # Cross-entropy cost
    m = A2.shape[0]
    logprobs = np.multiply(np.log(A2), Y) + np.multiply((1 - Y), np.log(1 - A2))
    cost = - np.sum(logprobs) / m
    
    cost = np.squeeze(cost)
    return cost

def back_prop(cache, parameters, X, Y): 
    A2 = cache['A2']
    Z2 = cache['Z2']
    A1 = cache['A1']
    Z1 = cache['Z1']
    
    W2 = parameters['W2']
    b2 = parameters['b2']
    W1 = parameters['W1']
    b1 = parameters['b1']
        
    
    m = X.shape[0]
    
    dZ2 = A2 - Y
    dW2 = (1/m)*np.dot(dZ2,A1.T)
    db2 = (1/m)*np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T, dZ2)*(1-np.tanh(Z1)**2)
    dW1 = (1/m)*np.dot(dZ1,X.T)
    db1 = (1/m)*np.sum(dZ1, axis=1, keepdims=True)
    
    grads = {'dW2':dW2,
             'db2':db2,
             'dW1':dW1,
             'db1':db1}
    
    return grads

def update_parameters(grads, parameters, learning_rate):
    W2 = parameters['W2']
    b2 = parameters['b2']
    W1 = parameters['W1']
    b1 = parameters['b1']
    
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

def neural_network(X, Y,num_iterations=10000):
    n_x, n_h, n_y = layer_size(X, Y)
    parameters = initialize_parameters(X, Y)

    for i in range(0, num_iterations):
        A2, cache = forward_prop(X, parameters)
        cost = cost_function(A2, Y)
        grads = back_prop(cache, parameters, X, Y)
        parameters = update_parameters(grads, parameters, learning_rate=0.005)
    return parameters

def predict(parameter, X):
    A2, cache = forward_prop(X, parameters)
    predictions = np.round(A2)    
    return predictions

In [7]:
iris_list = [setosa(), versicolor(), virginica()]
iris_list_names = [setosa, versicolor, virginica]
for flower_type in range(len(iris_list)):
    print('\nIris-', iris_list_names[flower_type])
    train_x,  test_x, train_y, test_y = iris_list[flower_type]
    parameters = neural_network(train_x, train_y)
    predictions_train = predict(parameters, train_x)
    predictions_test = predict(parameters, test_x)
    print('Train Accuracy: %d' % float((np.dot(train_y,predictions_train.T) + np.dot(1 - train_y, 1 - predictions_train.T)) / float(train_y.size) * 100) + '%')
    print('Test Accuracy: %d' % float((np.dot(test_y,predictions_test.T) + np.dot(1 - test_y, 1 - predictions_test.T)) / float(test_y.size) * 100) + '%')


Iris- <function setosa at 0x0000025A5E027BF8>
Train Accuracy: 100%
Test Accuracy: 100%

Iris- <function versicolor at 0x0000025A6189A378>
Train Accuracy: 98%
Test Accuracy: 96%

Iris- <function virginica at 0x0000025A6189A268>
Train Accuracy: 99%
Test Accuracy: 100%
